In [1]:
# Preprocess simulation data from Claudio. This is used for training.
# Author Peter
import h5py
import numpy as np

infile = '/mnt/lts/nfs_fs02/sadow_lab/shared/gcr/data/proposal/data.hdf5'
outfile = '/mnt/lts/nfs_fs02/sadow_lab/shared/gcr/data/proposal/data_processed.hdf5'

In [2]:
# Load, process, write data.
with h5py.File(infile, 'r') as f:
    print([k for k in f])
    print(f['info'])
    print(f['models'])
    dataset = f['models']
    # Remove events that have quality control flag.
    dataset = dataset[dataset['type'] == 0]
    
    X = np.vstack((dataset['alpha'], dataset['cmf'], dataset['cpa'], dataset['pwr1par'], dataset['pwr2par'], dataset['pwr1perr'], dataset['pwr2perr'])).T
    y = np.array(dataset['flux'])
    
print(X.shape, y.shape)

['info', 'models']
<HDF5 dataset "info": shape (1,), type "|V2102">
<HDF5 dataset "models": shape (1655808,), type "|V1018">
(1435308, 7) (1435308, 245)


In [4]:
# Write data to new file.
with h5py.File(outfile, 'w') as dest:
    dset = dest.create_dataset("features", data=X)
    dset = dest.create_dataset("flux", data=y)
    dset = dest.create_dataset("logp1_flux", data=np.log(y+1.))
    
# Write data to new file.
with h5py.File('./data_processed_sample.hdf5', 'w') as dest:
    dset = dest.create_dataset("features", data=X[:1000, ...])
    dset = dest.create_dataset("flux", data=y[:1000, ...])
    dset = dest.create_dataset("logp1_flux", data=np.log(1.+y[:1000, ...]))


In [30]:
# STatistics.
X[:].mean(axis=0), X[:].std(axis=0), X[:].max(axis=0), X[:].min(axis=0)
import matplotlib.pylab as plt
plt.hist(y[:10, :].flatten())

(1435308, 7)

In [36]:
# Example usage
import tensorflow_io as tfio # See https://stackoverflow.com/questions/60097970/tensorflow-io-dataset-input-pipeline-with-very-large-hdf5-files
BATCH_SIZE = 3
f = '/mnt/lts/nfs_fs02/sadow_lab/shared/gcr/data/proposal/data_processed.hdf5'
x = tfio.IODataset.from_hdf5(f, dataset='/features')
y = tfio.IODataset.from_hdf5(f, dataset='/flux')
full = tf.data.Dataset.zip((x, y)).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
train = full.take(np.floor(1435308/BATCH_SIZE *.6))
test = full.skip(np.floor(1435308/BATCH_SIZE *.6))

# Print 2 batches.
# for x, y in dataset.take(2).as_numpy_iterator():
#     print(x,y)